In [1]:
import tensorflow as tf
import pandas as pd
import cv2
import numpy as np
import os
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

csv_path = r"C:\Users\aryes\OneDrive\Desktop\vista-25\dataset\data_labels_cleaned.csv"
df = pd.read_csv(csv_path)

df['file_path'] = df['file_path'].astype(str)
df = df[df['file_path'].apply(os.path.exists)]

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

def preprocess_image(image_path, label):
    image_path = image_path.decode('utf-8')
    
    if not os.path.exists(image_path):
        print(f"Skipping missing image: {image_path}")
        return np.zeros((128, 128, 3), dtype=np.float32), np.float32(0)
    
    image = cv2.imread(image_path)
    if image is None:
        print(f"Skipping unreadable image: {image_path}")
        return np.zeros((128, 128, 3), dtype=np.float32), np.float32(0)
    
    image = cv2.resize(image, (128, 128))
    image = image.astype(np.float32) / 255.0
    return image, np.array(label, dtype=np.float32)

def create_tf_dataset(df, batch_size=32):
    image_paths = df['file_path'].values
    labels = df['label'].values.astype(np.float32)

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    def process_path(path, label):
        image, label = tf.numpy_function(
            func=preprocess_image, inp=[path, label], Tout=(tf.float32, tf.float32)
        )
        image.set_shape((128, 128, 3))
        label.set_shape(())
        return image, label

    dataset = dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = create_tf_dataset(train_df)
val_dataset = create_tf_dataset(val_df)

base_model = MobileNetV3Small(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
base_model.trainable = False

inputs = Input(shape=(128, 128, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

model.fit(train_dataset, validation_data=val_dataset, epochs=50, callbacks=[early_stopping])

base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.fit(train_dataset, validation_data=val_dataset, epochs=20, callbacks=[early_stopping])

model.save(r"C:\Users\aryes\OneDrive\Desktop\vista-25\saved_model_mobilenetv3.h5")


c:\Users\aryes\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\applications\mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/50
 115/1200 ━━━━━━━━━━━━━━━━━━━━ 11:09 617ms/step - accuracy: 0.5215 - loss: 0.7173 - precision: 0.5060 - recall: 0.5047Skipping unreadable image: C:\Users\aryes\OneDrive\Desktop\vista-25\dataset\train\real\real_11957.jpg
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 962s 700ms/step - accuracy: 0.5419 - loss: 0.6933 - precision: 0.5380 - recall: 0.5572 - val_accuracy: 0.6370 - val_loss: 0.6608 - val_precision: 0.6662 - val_recall: 0.5493
Epoch 2/50
 115/1200 ━━━━━━━━━━━━━━━━━━━━ 11:15 623ms/step - accuracy: 0.6376 - loss: 0.6526 - precision: 0.6438 - recall: 0.6242Skipping unreadable image: C:\Users\aryes\OneDrive\Desktop\vista-25\dataset\train\real\real_11957.jpg
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 923s 695ms/step - accuracy: 0.6184 - loss: 0.6574 - precision: 0.6268 - recall: 0.5804 - val_accuracy: 0.6457 - val_loss: 0.6399 - val_precision: 0.6532 - val_recall: 0.6210
Epoch 3/50
 115/1200 ━━━━━━━━━━━━━━━━━━━━ 10:57 606ms/step - accuracy: 0.6181 -

In [4]:
model.save(r"C:\Users\aryes\OneDrive\Desktop\vista-25\saved_model_mobilenetv3.keras")

In [5]:
import tensorflow as tf
import cv2
import numpy as np
import os
import pandas as pd
from tensorflow.keras.applications import MobileNetV3Small

# model = tf.keras.models.load_model(
#     r"C:\Users\aryes\OneDrive\Desktop\vista-25\saved_model_mobilenetv3.h5",
#     custom_objects={"MobileNetV3Small": MobileNetV3Small}
# )
model = tf.keras.models.load_model(r"C:\Users\aryes\OneDrive\Desktop\vista-25\saved_model_mobilenetv3.keras")


# model = tf.keras.models.load_model(r"C:\Users\aryes\OneDrive\Desktop\vista-25\saved_model_mobilenetv3.h5")

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Skipping unreadable image: {image_path}")
        return None
    
    image = cv2.resize(image, (128, 128))
    image = image.astype(np.float32) / 255.0
    return image

def test_and_save_results(test_dir, model, output_csv_path, batch_size=32):
    image_paths = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir) if fname.endswith(('.jpg', '.png'))]
    
    results = []
    
    for image_path in image_paths:
        image = preprocess_image(image_path)
        if image is None:
            continue
        
        image = np.expand_dims(image, axis=0)
        
        prediction = model.predict(image)
        predicted_label = 1 if prediction > 0.5 else 0
        predicted_label_str = 'Fake' if predicted_label == 1 else 'Real'
        
        results.append({
            'image_path': image_path,
            'prediction': predicted_label_str,
            'probability': prediction[0][0]
        })

    results_df = pd.DataFrame(results)

    results_df.to_csv(output_csv_path, index=False)
    print(f"Test results saved to {output_csv_path}")

test_dir = r"C:\Users\aryes\OneDrive\Desktop\vista-25\dataset\test"
output_csv_path = r"C:\Users\aryes\OneDrive\Desktop\vista-25\dataset\test_results_mobilenetv3.csv"
test_and_save_results(test_dir, model, output_csv_path)


c:\Users\aryes\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 57 variables whereas the saved optimizer has 112 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━

In [8]:
import pandas as pd
import re

# Load the test results CSV
file_path = r"C:\Users\aryes\OneDrive\Desktop\vista-25\dataset\test_results_mobilenetv3.csv"
df = pd.read_csv(file_path)

# Function to extract numerical image_id from image_path
def extract_image_id(image_path):
    match = re.search(r"(\d+)", image_path)  # Extracts the first numerical sequence
    return int(match.group(1)) if match else None

# Apply function to create 'image_id' column
df["image_id"] = df["image_path"].apply(extract_image_id)

# Keep only necessary columns and rename 'probability' to 'label'
submission_df = df[["image_id", "probability"]].rename(columns={"probability": "label"})

# Save the new submission CSV
submission_path = r"C:\Users\aryes\OneDrive\Desktop\vista-25\new_submission_mobilenetv3.csv"
submission_df.to_csv(submission_path, index=False)

print(f"Submission file saved to {submission_path}")


Submission file saved to C:\Users\aryes\OneDrive\Desktop\vista-25\new_submission_mobilenetv3.csv
